# Project: Performance Evaluation of Bandit Algorithms

- In this project, you will implement several classical bandit algorithms, evluate their performance via numerical comparison and finally gain inspiring intuition.

## Part I: Classical Bandit Algorithms

We consider a time-slotted bandit system ($t=1,2,\ldots$) with three arms.
We denote the arm set as $\{1,2,3\}$.
Pulling each arm $j$ ($ j \in \{1,2,3\}$) will obtain a random reward $r_{j}$, which follows a Bernoulli distribution with mean $\theta_{j}$, *i.e.*, Bern($\theta_{j}$).
Specifically,

\begin{equation*}
	\begin{aligned}
		r_{j} = 
		\begin{cases}
			1, & w.p.\ \theta_{j}, \\
			0, & w.p.\ 1-\theta_{j},			
		\end{cases}
	\end{aligned}
\end{equation*}
where $\theta_{j}, j \in\{1,2,3\}$ are parameters within $(0,1)$.
  
Now we run this bandit system for $N$ ($N \gg 3$) time slots.
In each time slot $t$, we choose one and only one arm from these three arms, which we denote as $I(t) \in \{1,2,3\}$.
Then we pull the arm $I(t)$ and obtain a random reward $r_{I(t)}$.
Our objective is to find an optimal policy to choose an arm $I(t)$ in each time slot $t$ such that the expectation of the aggregated reward over $N$ time slots is maximized, *i.e.*,

\begin{equation*}
	\begin{aligned}
		\max_{I(t),t = 1,\dots,N} \ \  \mathbb{E}\left[\sum_{t=1}^{N} r_{I(t)} \right].
	\end{aligned}  	
\end{equation*}

If we know the values of $\theta_{j},j \in \{1,2,3\}$, this problem is trivial.
Since $r_{I(t)} \sim \text{Bern}(\theta_{I(t)})$,

\begin{equation*}
	\begin{aligned}
		\mathbb{E}\left[\sum_{t=1}^N r_{I(t)} \right] 
		= \sum_{t=1}^{N} \mathbb{E}[r_{I(t)}] 
		= \sum_{t=1}^N \theta_{I(t)}.
	\end{aligned} 	
\end{equation*}

Let $I(t) = I^{*} = \mathop{\arg \max}\limits_{ j \in \{1,2,3\}} \ \theta_j$ for $t=1,2,\ldots,N$, then 

\begin{equation*}
	\begin{aligned}
		\max_{I(t),t=1,\ldots,N} \ \  \mathbb{E}\left[\sum_{t=1}^N r_{I(t)} \right] = N \cdot \theta_{I^*}.
	\end{aligned} 	
\end{equation*}

However, in reality, we do not know the values of $\theta_{j},j \in \{1,2,3\}$.
We need to estimate the values $\theta_{j}, j \in \{1,2,3\}$ via empirical samples, and then make the decisions in each time slot. 
Next we introduce three classical bandit algorithms: $\epsilon$-greedy, UCB, and TS, respectively.

### $\epsilon$-greedy Algorithm ($0 \leq \epsilon \leq 1$)
<img src="figures/e-greedy.jpg" width="50%" align='left'>

### UCB (Upper Confidence Bound) Algorithm
<img src="figures/UCB.jpg" width="50%" align='left'>

### TS (Thompson Sampling) Algorithm
<img src="figures/TS.jpg" width="50%" align='left'>

### Problems

1. Now suppose we obtain the parameters of the Bernoulli distributions from an oracle, which are shown in the following table. Choose $N=5000$ and compute the theoretically maximized expectation of aggregate rewards over $N$ time slots. We call it the oracle value. Note that these parameters $\theta_{j}, j \in \{1,2,3\}$ and oracle values are unknown to all bandit algorithms.

| Arm $j$ | 1   | 2   | 3   |
|---------|-----|-----|-----|
| $\theta_j$ | 0.7 | 0.5 | 0.4 |

**Your anwser of problem 1 in Part I**

Since the value of $\theta_j$ is given, then we just have to choose the Arm 1 for we have a probability of getting reward to $0.7$, which is the maximum value among the three arms. Thus, the expectation of aggregate rewards over $5000$ slots would be 
$$N\cdot \theta_1 = 0.7 \cdot 5000 = 3500 $$

2. Implement aforemented three classical bandit algorithms with following settings: 
   
	- $N=5000$
	- $\epsilon$-greedy with $\epsilon \in \{0.1, 0.5, 0.9\}$.
	- UCB with $c \in \{1,5,10\}$.
	- TS with
    	- $\left\{(\alpha_1,\beta_1)=(1,1),(\alpha_2,\beta_2)=(1,1),(\alpha_3,\beta_3)=(1,1)\right\}$ 
    	- $\left\{(\alpha_1,\beta_1)=(601,401),(\alpha_2,\beta_2)=(401,601),(\alpha_3,\beta_3)=(2,3)\right\}$

**Your anwser of problem 2 in Part I**

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random, math, copy
### Import more packages if you need

In [11]:
### Feel free to insert more blocks or helper functions if you need.

In [12]:
### Implementation of epsilon-Greedy:
### n is the number of time slots, epsilon is the parameter of the algorithm
### return the total reward
def greedy(n, epsilon) -> tuple[int, np.ndarray]:
    theta = np.array([0.7, 0.5, 0.4], dtype=float)   # used to generate rewards
    theta_guess = np.array([0, 0, 0], dtype=float)  #initilize the vector we will adjust
    count = np.array([0, 0, 0], dtype=int)  #init
    total_r = 0   # this the sum of all rewards
    for t in range(0, n):
        choice = np.random.uniform(0, 1)    # used to choose I(t)
        # arm is I(t)
        arm = np.argmax(theta_guess) if choice > epsilon else np.random.choice([0, 1, 2])
        count[arm] += 1
        r = np.random.binomial(1, theta[arm])   # generate the reward
        theta_guess[arm] += 1 / count[arm] * (r - theta_guess[arm])
        total_r += r
    return total_r, theta_guess

In [13]:
### Implementation of UCB Algorithm:
### n is the number of time slots, c is the parameter of the algorithm
### return the total reward
def UCB(n, c) -> tuple[int, np.ndarray]:
    theta = np.array([0.7, 0.5, 0.4], dtype=float)  # used to generate rewards
    theta_guess = np.array([0, 0, 0], dtype=float)
    for i in range(3):
        theta_guess[i] = np.random.binomial(1, theta[i])    # initilize the thetas
    count = np.array([1, 1, 1], dtype=int)
    total_r = 0 
    for t in range(4, n+1):
        # arm is the I(t)
        arm = np.argmax(theta_guess + c * np.sqrt(2 * np.log(t) / count))
        count[arm] += 1
        r = np.random.binomial(1, theta[arm])   # reward
        theta_guess[arm] += 1 / count[arm] * (r - theta_guess[arm])
        total_r += r
    return total_r, theta_guess

In [14]:
### Implementation of TS Algorithm
### n is the number of time slots, a and b are the parameters of the algorithm
### return the total reward

# a = [a1, a2, a3] and b = [b1, b2, b3]
def TS(n, a, b) -> tuple[int, np.ndarray]:
    theta = np.array([0.7, 0.5, 0.4], dtype=float)
    beta = np.array([(a[0], b[0]), (a[1], b[1]), (a[2], b[2])], dtype=int)
    total_r = 0
    for t in range(0, n):
        # use this vector to store the expectation of beta distributions
        E_theta = np.array([0, 0, 0], dtype=float)  
        for j in range(3):
            E_theta[j] = beta[j][0]/(beta[j][0] + beta[j][1])
        arm = np.argmax(E_theta)
        r = np.random.binomial(1, theta[arm]) # reward
        beta[arm][0] += r
        beta[arm][1] += 1- r
        total_r += r
    return total_r, beta

3. Regard each of the above setting in problem 2 of Part I as an experiment (in total $8$ experiments).
Run each experiment $200$ independent trials (change the random seed).
Plot the final result (in terms of rewards and regrets) averaged over these $200$ trials.

**Your anwser of problem 3 in Part I**

In [15]:
num_trials = 200

In [21]:
## greedy
epsilons = [0.1, 0.5, 0.9]
total_rs = np.array([0, 0, 0], dtype=int)
thetas = np.zeros([3, 3], dtype=float) 
for k in range(3):
    for i in range(num_trials):
        r, theta = greedy(5000, epsilons[k])
        total_rs[k] += r
        thetas[k] += theta
greedy_rewards = total_rs / num_trials
greedy_thetas = thetas / num_trials
for i in range(3):
    print("When epsilon = {}, the averaged reward is {:.6f}, thetas are {}" .format(epsilons[i], greedy_rewards[i], greedy_thetas[i]))

When epsilon = 0.1, the averaged reward is 3412.555000, thetas are [0.69963108 0.49448575 0.40367482]
When epsilon = 0.5, the averaged reward is 3080.555000, thetas are [0.69996539 0.49876338 0.39977516]
When epsilon = 0.9, the averaged reward is 2750.300000, thetas are [0.70079345 0.50073627 0.39887706]


In [23]:
## UCB
c = [1, 5, 10]
total_rs = np.array([0, 0, 0], dtype=int)
thetas = np.zeros([3, 3], dtype=float) 
for k in range(3):
    for i in range(num_trials):
        r, theta = UCB(5000, c[k])
        total_rs[k] += r
        thetas[k] += theta
ucb_rewards = total_rs / num_trials
ucb_thetas = thetas / num_trials
for i in range(3):
    print("When c = {}, the averaged reward is {:.6f}, thetas are {}" .format(c[i], ucb_rewards[i], ucb_thetas[i]))

When c = 1, the averaged reward is 3405.170000, thetas are [0.69914628 0.49002058 0.38650281]
When c = 5, the averaged reward is 2976.625000, thetas are [0.69965383 0.49831795 0.39716002]
When c = 10, the averaged reward is 2826.720000, thetas are [0.69956058 0.50176087 0.39950346]


In [36]:
## Thompson
a = np.array([[1, 1, 1], [601, 401, 2]])
b = np.array([[1, 1, 1], [401, 601, 3]])
total_rs = np.array([0, 0], dtype=int)
betas = np.zeros([2, 3, 2], dtype=float)
for k in range(2):
    for i in range(num_trials):
        r, beta = TS(5000, a[k], b[k])
        total_rs[k] += r
        betas[k] += beta
ts_rewards = total_rs / num_trials
ts_betas = betas / num_trials
ts_thetas = np.zeros([2, 3], dtype=float)
for i in range(2):
    for j in range(3):
        ts_thetas[i][j] = ts_betas[i][j][0] / (ts_betas[i][j][0] + ts_betas[i][j][1])
print("When (a1,b1) = (1,1), (a2,b2) = (1,1), (a3,b3) = (1,1), the total reward is {:.4f},\n the thetas are {}" 
      .format(ts_rewards[0], ts_thetas[0]))
print("When (a1,b1) = (601,401), (a2,b2) = (401,601), (a3,b3) = (2,3), the total reward is {:.4f},\n the thetas are {}" 
      .format(ts_rewards[1], ts_thetas[1]))

When (a1,b1) = (1,1), (a2,b2) = (1,1), (a3,b3) = (1,1), the total reward is 3344.9350,
 the thetas are [0.699717   0.49715673 0.41179002]
When (a1,b1) = (601,401), (a2,b2) = (401,601), (a3,b3) = (2,3), the total reward is 3498.3200,
 the thetas are [0.68299234 0.4001996  0.4       ]


4. Compute the gaps between the algorithm outputs (aggregated rewards over $N$ time slots) and the oracle value. Compare the numerical results of $\epsilon$-greedy, UCB, and TS.
   - Which one is the best?
   - Discuss the impacts of $\epsilon$, $c$, and $\alpha_{j}$, $\beta_{j}$, respectively. 

**Your anwser of problem 4 in Part I**

In [ ]:
### Your code for problem 1.4. Feel free to insert more blocks or helper functions if you need.

5. Give your understanding of the exploration-exploitation trade-off in bandit algorithms.

**Your anwser of problem 5 in Part I**

6. We implicitly assume the reward distribution of these three arms are independent. How about the dependent case?
	Can you design an algorithm to exploit such information to obtain a better result?

**Your anwser of problem 6 in Part I**

In [ ]:
### Your code for problem 1.6. Feel free to insert more blocks or helper functions if you need.

## Part II: Bayesian Bandit Algorithms

There are two arms which may be pulled repeatedly in any order.
Each pull may result in either a success or a failure.
The sequence of successes and failures which results from pulling arm $i$ ($i \in \{1, 2\}$) forms a Bernoulli process with unknown success probability $\theta_{i}$.
A success at the $t^{th}$ pull yields a reward $\gamma^{t-1}$ ($0 < \gamma <1$), while an unsuccessful pull yields a zero reward.
At time zero, each $\theta_{i}$ has a Beta prior distribution with two parameters $\alpha_{i}, \beta_{i}$ and these distributions are independent for different arms.
These prior distributions are updated to posterior distributions as arms are pulled.
Since the class of Beta distributions is closed under Bernoulli sampling, posterior distributions are all Beta distributions.
How should the arm to pull next in each time slot be chosen to maximize the total expected reward from an infinite sequence of pulls?

1. 	One intuitive policy suggests that in each time slot we should pull the arm for which the current expected value of $\theta_{i}$ is the largest.
	This policy behaves very good in most cases.
	Please design simulations to check the behavior of this policy.

**Your anwser of problem 1 in Part II**

In [1]:
### Your code for problem 2.1. Feel free to insert more blocks or helper functions if you need.

2. However, such intuitive policy is unfortunately not optimal.
	Please provide an example to show why such policy is not optimal. 

**Your anwser of problem 2 in Part II**

3. For the expected total reward under an optimal policy, show that the following recurrence equation holds:

\begin{equation*}
		\begin{aligned}
			R_{1}(\alpha_{1},\beta_{1}) 
			= & \frac{\alpha_{1}}{\alpha_{1}+\beta_{1}} [1+\gamma R(\alpha_{1} + 1, \beta_{1}, \alpha_{2}, \beta_{2})] \\
				& + \frac{\beta_{1}}{\alpha_{1} + \beta_{1}} [\gamma R(\alpha_{1}, \beta_{1} + 1, \alpha_{2}, \beta_{2})]; \\
			R_{2}(\alpha_{2}, \beta_{2}) 
			= & \frac{\alpha_{2}}{\alpha_{2} + \beta_{2}} [1 + \gamma R(\alpha_{1}, \beta_{1}, \alpha_{2} + 1, \beta_{2})] \\
				& + \frac{\beta_{2}}{\alpha_{2} + \beta_{2}} [\gamma R(\alpha_{1}, \beta_{1}, \alpha_{2}, \beta_{2} + 1)]; \\
			R(\alpha_{1}, \beta_{1}, \alpha_{2}, \beta_{2}) 
			= & \max \left\{ R_{1}(\alpha_{1}, \beta_{1}), R_{2}(\alpha_{2}, \beta_{2}) \right\}.
		\end{aligned}  	
	\end{equation*}

**Your anwser of problem 3 in Part II**

4. For the above equations, how to solve it exactly or approximately? 

**Your anwser of problem 4 in Part II**

In [ ]:
### Your code for problem 2.4 if needed.

5. Find the optimal policy.

**Your anwser of problem 5 in Part II**

In [ ]:
### Your code for problem 2.5. Feel free to insert more blocks or helper functions if you need.